# Coursera Week 3 Part 1

In [53]:
import pandas as pd
import numpy as np
print("Importing complete")

Importing complete


Reading table

In [54]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [55]:
df['Borough'].value_counts()

Not assigned        77
Etobicoke           44
North York          38
Scarborough         37
Downtown Toronto    37
Central Toronto     17
West Toronto        13
York                 9
East Toronto         7
East York            6
Queen's Park         1
Mississauga          1
Name: Borough, dtype: int64

In [56]:
df['Neighbourhood'].value_counts()

Not assigned                78
Runnymede                    2
St. James Town               2
Sullivan                     1
North Park                   1
West Deane Park              1
Newtonbrook                  1
L'Amoreaux East              1
Islington                    1
Weston                       1
Martin Grove Gardens         1
Kingsview Village            1
Emery                        1
Woburn                       1
King                         1
Cedarbrae                    1
Kingsway Park South West     1
Oakridge                     1
Downsview West               1
Don Mills South              1
Humber Bay Shores            1
CN Tower                     1
Markland Wood                1
King's Mill Park             1
Morningside                  1
Cloverdale                   1
Forest Hill North            1
Kingsway Park South East     1
Grange Park                  1
Humberlea                    1
                            ..
Martin Grove                 1
Del Ray 

## Clean 'Borough' column

In [57]:
df['Borough'].replace('Not assigned',np.nan,inplace=True)
df.dropna(inplace=True)
df.reset_index(drop=True,inplace=True)
df['Borough'].value_counts()

Etobicoke           44
North York          38
Scarborough         37
Downtown Toronto    37
Central Toronto     17
West Toronto        13
York                 9
East Toronto         7
East York            6
Queen's Park         1
Mississauga          1
Name: Borough, dtype: int64

## Clean 'Neighbourhood' column

In [58]:
#this finds nameless neighbourhoods
df['Neighbourhood'].replace('Not assigned',np.nan,inplace=True)
df[df.Neighbourhood.isna()]

,Postcode,Borough,Neighbourhood
5,M7A,Queen's Park,NaN


In [59]:
#this renames the nameless neighbourhoods with their borough names
df.at[5,'Neighbourhood']=df.at[5,'Borough']
df.loc[5]

Postcode                  M7A
Borough          Queen's Park
Neighbourhood    Queen's Park
Name: 5, dtype: object

## Collapse neighbourhoods into zipcodes

In [60]:
#this code groups the neighborhoods by postcode, then concatenates the neighborhoods
df1=df
df2 = pd.DataFrame(columns = df1.columns)
ix = 0
for key,value in df1.groupby(['Postcode']).groups.items():
    ns=''
    b = df1.loc[value[0]][1]
    for indexdf in value:
        n = df1.loc[indexdf][2]
        ns+= n + ", "
    ns=ns.rstrip(", ")
    df2.loc[ix] = [key,b,ns]
    ix+=1
    

df2.head()         

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [61]:
df2.shape

(103, 3)

# Coursera Week 3 Part 2

Geocoder was too unreliable, below is my code for my initial try, followed by reading in the CSV

In [62]:
#Geocoder too unreliable
#!conda install -c conda-forge geocoder --yes
#import geocoder 

In [63]:
#This is how I'd try if geocoder worked
#new dataframe to merge later
'''df3 = pd.DataFrame(columns = ["Latitude","Longitude"])
df3.head()
for i in range(df2.shape[0]):
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    count = 0
    while(lat_lng_coords is None):
        count+=1
        g = geocoder.google('{}, Toronto, Ontario'.format(df2.iloc[i][0]))
        lat_lng_coords = g.latlng
    print("Row " + str(i) + " took " + str(count) + " tries.")
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    #update dataframe
    df3.loc[i]=[latitude,longitude]
df3.head() '''  

'df3 = pd.DataFrame(columns = ["Latitude","Longitude"])\ndf3.head()\nfor i in range(df2.shape[0]):\n    # initialize your variable to None\n    lat_lng_coords = None\n\n    # loop until you get the coordinates\n    count = 0\n    while(lat_lng_coords is None):\n        count+=1\n        g = geocoder.google(\'{}, Toronto, Ontario\'.format(df2.iloc[i][0]))\n        lat_lng_coords = g.latlng\n    print("Row " + str(i) + " took " + str(count) + " tries.")\n    latitude = lat_lng_coords[0]\n    longitude = lat_lng_coords[1]\n    #update dataframe\n    df3.loc[i]=[latitude,longitude]\ndf3.head() '

In [64]:
#Reading in provided csv worked flawlessly
df3 = pd.read_csv("Geospatial_Coordinates.csv")
df3.columns = ["Postcode","Latitude","Longitude"]
df3.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Part 2 final dataframe

In [65]:
df3=df3.merge(df2,on="Postcode")
df3.head()

,Postcode,Latitude,Longitude,Borough,Neighbourhood
0,M1B,43.806686,-79.194353,Scarborough,"Rouge, Malvern"
1,M1C,43.784535,-79.160497,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae


# Coursera Week 3 Part 3

Explore and cluster neighborhoods in Toronto.  Maps included.

In [73]:
#!conda install -c conda-forge folium=0.5.0 --yes 
import folium
import requests
import json
df4=df3
CLIENT_ID = '3B3T4ZAT2ADVQ15YL0QJ3H2DUFNYLBQ3W4J4KJWI5WDVG0PE' # your Foursquare ID
CLIENT_SECRET = 'YGXJYCHEJ410LTZKYP5FGQF4C1GQTOBTGMP0EJC1YTINVTSF' # your Foursquare Secret
VERSION = '20180604'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

torll = [43.6487,-79.38544]

Your credentails:
CLIENT_ID: 3B3T4ZAT2ADVQ15YL0QJ3H2DUFNYLBQ3W4J4KJWI5WDVG0PE
CLIENT_SECRET:YGXJYCHEJ410LTZKYP5FGQF4C1GQTOBTGMP0EJC1YTINVTSF


## Create map of neighborhoods for visualization


In [ ]:
# create map of Manhattan using latitude and longitude values
map_tor = folium.Map(location=torll, zoom_start=10)

# add markers to map
for lat, lng, label in zip(df3['Latitude'], df3['Longitude'], df3['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tor)  
    
map_tor

## Foursquare exploration

In [130]:
#taking a look at how data is extracted for some item in our dataset
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, df3["Latitude"][11], df3["Longitude"][11], VERSION, 500, 20)
results = requests.get(url).json()
results=results['response']['groups'][0]['items']
results[0]['venue']['location']

{'lat': 43.746098,
 'lng': -79.293142,
 'labeledLatLngs': [{'label': 'display', 'lat': 43.746098, 'lng': -79.293142}],
 'distance': 492,
 'cc': 'CA',
 'country': 'Canada',
 'formattedAddress': ['Canada']}

 This loop calls foursquare for each post code and then lists the top 20 (if possible) venues within 1 km.

In [145]:
venuesdf = pd.DataFrame(columns = ["Postcode","Neighbourhood","Name","Category","vlat",'vlong'])
radius = 1000
lim=50
count=0
for ix in range(df4.shape[0]):
    
    [post,nlat,nlong,b,n] = df4.loc[ix]
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, nlat, nlong, VERSION, radius, lim)
    results = requests.get(url).json()['response']['groups'][0]['items']
    for venue in results:
        temp = venue['venue']
        venuesdf.loc[count]=[post,n,temp['name'],temp['categories'][0]['name'],temp['location']['lat'],temp['location']['lng']]
        count+=1
venuesdf.head()

,Postcode,Neighbourhood,Name,Category,vlat,vlong
0,M1B,"Rouge, Malvern",Images Salon & Spa,Spa,43.802283,-79.198565
1,M1B,"Rouge, Malvern",Caribbean Wave,Caribbean Restaurant,43.798558,-79.195777
2,M1B,"Rouge, Malvern",Wendy's,Fast Food Restaurant,43.802008,-79.198080
3,M1B,"Rouge, Malvern",Wendy's,Fast Food Restaurant,43.807448,-79.199056
4,M1B,"Rouge, Malvern",Staples Morningside,Paper / Office Supplies Store,43.800285,-79.196607


In [ ]:
#looking at trends in summary data
venuesdf.shape
venuesdf.groupby('Postcode').count()
#venuesdf.groupby('Category').count()

In [172]:
venuesdf2 = pd.get_dummies(venuesdf['Category'], prefix="", prefix_sep="")
venuesdf2['Neighbourhood'] = venuesdf.Neighbourhood

In [173]:

cols = venuesdf2.columns.tolist()
cols.reverse()
venuesdf3 = venuesdf2[cols]
venuesdf3.head()

,Neighbourhood,Zoo,Yoga Studio,Women's Store,Wings Joint,Wine Shop,Wine Bar,Warehouse Store,Vietnamese Restaurant,Video Store,...,Antique Shop,Animal Shelter,Amphitheater,American Restaurant,Airport Lounge,Airport,African Restaurant,Afghan Restaurant,Accessories Store,ATM
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [175]:
venuesdf3.groupby('Neighbourhood').mean().reset_index()

,Neighbourhood,Zoo,Yoga Studio,Women's Store,Wings Joint,Wine Shop,Wine Bar,Warehouse Store,Vietnamese Restaurant,Video Store,...,Antique Shop,Animal Shelter,Amphitheater,American Restaurant,Airport Lounge,Airport,African Restaurant,Afghan Restaurant,Accessories Store,ATM
0,"Adelaide, King, Richmond",0.00,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,...,0.0,0.0,0.00,0.040000,0.000000,0.000000,0.000000,0.00,0.0,0.0
1,Agincourt,0.00,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.020000,0.000000,...,0.0,0.0,0.00,0.020000,0.000000,0.000000,0.000000,0.00,0.0,0.0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.00,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,...,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.00,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,...,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0
4,"Alderwood, Long Branch",0.00,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,...,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0
5,"Bathurst Manor, Downsview North, Wilson Heights",0.00,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,...,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0
6,Bayview Village,0.00,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,...,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0
7,"Bedford Park, Lawrence Manor East",0.00,0.000000,0.000000,0.027778,0.0,0.00,0.00,0.000000,0.027778,...,0.0,0.0,0.00,0.027778,0.000000,0.000000,0.000000,0.00,0.0,0.0
8,Berczy Park,0.00,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,...,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0
9,"Birch Cliff, Cliffside West",0.00,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,...,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0


# Testing cell

In [74]:
df3["Longitude"][0]

-79.19435340000001

In [127]:
for ix,row in df4.iterrows():
    print(row)

Postcode                    M1B
Latitude                43.8067
Longitude              -79.1944
Borough             Scarborough
Neighbourhood    Rouge, Malvern
Name: 0, dtype: object
Postcode                                            M1C
Latitude                                        43.7845
Longitude                                      -79.1605
Borough                                     Scarborough
Neighbourhood    Highland Creek, Rouge Hill, Port Union
Name: 1, dtype: object
Postcode                                       M1E
Latitude                                   43.7636
Longitude                                 -79.1887
Borough                                Scarborough
Neighbourhood    Guildwood, Morningside, West Hill
Name: 2, dtype: object
Postcode                 M1G
Latitude              43.771
Longitude           -79.2169
Borough          Scarborough
Neighbourhood         Woburn
Name: 3, dtype: object
Postcode                 M1H
Latitude             43.7731
Longitude 